# Credit Card Fraud Detection Project

In [1]:
# Import relevant modules
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Load the dataset into pandas DF
credit_card_data = pd.read_csv("/Users/jeremy.yap/Documents/Data Science/Portfolio Projects/Credit Card Fraud Detection Project - Python Machine Learning/creditcard.csv")
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [13]:
# Check how many rows and columns
credit_card_data.shape

(284807, 31)

In [10]:
# Check any missing value in the dataset
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [17]:
# Assuming Class = 0 is legit transaction and Class = 1 is fraudulent transaction
# Check how many fraudulent transaction are there
credit_card_data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

##### As above, we can see that the dataset is highly unbalanced which is not good for machine learning process

In [14]:
# Separate the dataset for analysis
legit = credit_card_data[credit_card_data['Class'] == 0]
fraud = credit_card_data[credit_card_data['Class'] == 1]
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [15]:
# Understand the statistical measures of the normal transaction data
legit['Amount'].describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [16]:
# Understand the statistical measures of the fraudulent transaction data
fraud['Amount'].describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

##### Both transactions have the shape of distribution that skewed to the right (Mean > Median)

In [18]:
# Compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


##### We can see that there is wide difference between the mean of fraudulent transaction and the normal transaction
##### Machine learning algorithm can predict whether the transaction is legit or fraudulent

In [19]:
# Build a sample dataset containing similar distribution of normal transaction and fraudulent transaction
# We know that the number of fraudulent transaction = 492
# Sampling the normal dataset randomly
legit_sample = legit.sample(n= 492)

In [22]:
# Combining the two equal datasets using concatenating
new_dataset = pd.concat([legit_sample, fraud], axis= 0)
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
28016,34811.0,0.775570,-0.770648,0.936321,1.312951,-1.060369,0.343982,-0.509566,0.281372,0.823800,...,0.064923,-0.121614,-0.206049,-0.029554,0.318518,-0.420916,0.025195,0.054102,194.00,0
161590,114345.0,-0.512730,0.931456,-0.272783,1.063161,1.285259,0.011887,1.301499,-0.466786,0.573212,...,-0.145688,0.620170,-0.125644,0.341808,0.179209,-0.390911,0.321728,-0.102203,63.96,0
73725,55253.0,-1.266551,-0.346458,1.712679,-0.174390,-0.518872,0.985611,0.415991,0.062594,-1.152437,...,-0.199128,0.417235,0.129288,-0.334492,-0.416469,-0.349661,0.180617,-0.035787,176.50,0
161602,114356.0,0.018692,0.800663,0.173847,-0.786544,0.647281,-0.524501,0.891080,-0.012588,-0.279455,...,-0.225081,-0.497739,0.023578,-0.403217,-0.491604,0.146405,0.245974,0.083208,3.57,0
280281,169427.0,-1.426486,1.493432,0.647779,1.226616,0.373659,-0.549780,0.592911,0.189688,-0.234821,...,-0.107435,-0.104038,-0.364264,-0.223957,0.620470,-0.395340,-0.176356,-0.077509,39.00,0


In [24]:
# Check whether the new datasets containing equal amount of data on indivudal class
new_dataset['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [25]:
# Check the distribution of the new dataset
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95605.993902,0.034498,0.023395,0.010890,0.013470,0.019269,0.040458,0.088796,0.005418,0.045666,...,0.020954,0.054703,-0.051572,0.001467,-0.001445,0.027418,-0.032629,0.009681,0.003000,95.151768
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


##### The mean difference between the two classes are similar to the origin dataset
##### Therefore, the new dataset is considered a good sample

In [26]:
# Splitting the data into features & targets
X = new_dataset.drop(columns= 'Class', axis= 1)
Y = new_dataset['Class']

In [28]:
# Split the data into training data & testing data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, stratify= Y, random_state= 2)

In [29]:
# Check the training data after splitting
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


In [30]:
 # Check the testing data after splitting
print(Y.shape, Y_train.shape, Y_test.shape)


(984,) (787,) (197,)


In [34]:
# Train the logistic regression model with training data
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [38]:
# Find the accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [39]:
print('Accuracy on training data: ', training_data_accuracy)

Accuracy on training data:  0.9263024142312579


In [40]:
# Find the accuracy on testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)


In [41]:
print('Accuracy on testing data: ', testing_data_accuracy)

Accuracy on testing data:  0.9086294416243654


##### In conclusion, the accuracy score on both training data and testing data are close to each other,the model is not overfitting or underfitting. 